In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import os
import subprocess
import sys
import datetime
from typing import Any, Dict, List, Optional, Type, Union
from datetime import datetime
import pandas as pd
import os
import sys
import numpy as np
import importlib
from concurrent.futures import ThreadPoolExecutor

from pys.data_collection.bonds_parser import MOEXBondHistoricalParser


In [2]:
bond_parser = MOEXBondHistoricalParser()

result = bond_parser.parse_from_2024_to_now(
    include_non_trading_days=True,  # Включаем выходные дни
    analyze_continuity=True,        # Анализируем непрерывность и создаем полный датасет
    yield_range=(0, 100),           # Широкий диапазон - берем все
    price_range=(0, 100),           # Широкий диапазон - берем все
    duration_range=(0, 7200)         # Широкий диапазон - берем все
)

print(f"Всего дней с данными: {len(result)}")
trading_days = sum(1 for date in result.keys() if datetime.strptime(date, '%Y-%m-%d').weekday() < 5)
print(f"Торговых дней: {trading_days}")
print(f"Нерабочих дней с данными: {len(result) - trading_days}")



2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - MOEX Bond Parser initialized
2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - Запуск массового парсинга данных с 2024-01-01 по 2025-04-24
2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - Запрошен период с 2024-01-01 по 2025-04-24
2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - Получение облигаций за период с 2024-01-01 по 2025-04-24
2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - Обработка даты: 2024-01-01 (рабочий день)
2025-04-25 23:37:05 - MOEXBondHistoricalParser - INFO - Получение списка облигаций на 2024-01-01


KeyboardInterrupt: 

In [4]:
optimal_threshold = bond_parser.find_optimal_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    min_bonds=20,
    max_threshold=99
)

print(f"Оптимальный порог непрерывности: {optimal_threshold}%")

dataset = bond_parser.generate_dataset_with_threshold(
    start_date=datetime(2024, 1, 1),
    end_date=datetime(2025, 4, 22),
    continuity_threshold=optimal_threshold
)

if dataset is not None:
    print(f"Создан датасет с {len(dataset['secid'].unique())} облигациями и {len(dataset['date'].unique())} датами")
else:
    print("Не удалось создать датасет")

2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Поиск оптимального порога непрерывности для периода с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Анализ непрерывности данных по облигациям с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 342 рабочих дней в указанном периоде
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 340 дней с доступными данными


2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Всего уникальных облигаций: 355
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Облигаций с непрерывными данными: 0 (0.00%)
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Результаты анализа сохранены в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/continuity_analysis_20240101_20250422
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - При пороге 99%: доступно 49 облигаций
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найден оптимальный порог: 99%
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Генерация датасета с порогом непрерывности 99% за период с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Анализ непрерывности данных по облигациям с 2024-01-01 по 2025-04-22
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 342 рабочих дней в указанном периоде
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 340 дне

Оптимальный порог непрерывности: 99%


2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Всего уникальных облигаций: 355
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Облигаций с непрерывными данными: 0 (0.00%)
2025-04-25 17:38:44 - MOEXBondHistoricalParser - INFO - Найдено 49 облигаций с порогом непрерывности не менее 99%
2025-04-25 17:38:45 - MOEXBondHistoricalParser - INFO - Датасет с порогом непрерывности 99% создан и сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/threshold_dataset_20240101_20250422_99
2025-04-25 17:38:45 - MOEXBondHistoricalParser - INFO - Всего дат: 340, облигаций: 49


Создан датасет с 49 облигациями и 339 датами


In [1]:
import sys
import importlib

from pys.data_collection.private_info import BASE_PATH
from pys.data_collection.bonds_processor import run_pipeline_bonds_processor

In [17]:
results = run_pipeline_bonds_processor(
    base_path=BASE_PATH,
    # Дополнительные параметры можно указать при необходимости
    # min_yield=7.0,
    # max_duration=4.0,
    # n_bonds=7
)


2025-04-25 17:53:17 - BondsProcessor - INFO - Loaded previous recommendations from 2025-04-25
2025-04-25 17:53:17 - BondsProcessor - INFO - Loading direct data from /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/moex/threshold_dataset_20240101_20250422_99/bonds_dataset.csv
2025-04-25 17:53:17 - BondsProcessor - INFO - Renamed columns: {'secid': 'security_code', 'name': 'full_name', 'price': 'price_pct', 'duration': 'duration_months', 'maturity_date': 'expiration_date', 'volume': 'trading_volume', 'value': 'trading_value'}
2025-04-25 17:53:17 - BondsProcessor - INFO - Created duration_years column from duration_months
2025-04-25 17:53:17 - BondsProcessor - INFO - Created snapshot_date from date column
2025-04-25 17:53:17 - BondsProcessor - INFO - Filtered data to use only latest snapshot: 2025-04-22 00:00:00
2025-04-25 17:53:17 - BondsProcessor - INFO - Starting filters with 49 bonds
2025-04-25 17:53:17 - BondsProcessor - INFO - After min_yield filter: 14 bonds
2025-04

Обработано 10 облигаций

=== Рекомендованный облигационный портфель ===
      security_code   full_name  yield  duration_years  weight
16726  RU000A0JUAH8  РЖД-30 обл  20.63        0.510833  0.3305
16696  KG000A3LSJ06  Кыргыз. 02  20.77        0.658333  0.2565
16709  RU000A0JSGV0  РЖД-32 обл  20.96        1.135833  0.1487
16712  RU000A0JT6B2   ВЭБ.РФ 19  19.91        0.922500  0.1830
16711  RU000A0JT403   ВЭБ.РФ 18  20.11        2.077500  0.0813

Портфель сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/results/bond_portfolio_20250425.csv
Визуализация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/results/bond_universe_20250425.png


2025-04-25 17:53:18 - BondsProcessor - INFO - Portfolio analysis generated successfully



=== Характеристики портфеля ===
Средневзвешенная доходность: 20.54%
Средневзвешенная дюрация: 0.84 лет
Уровень риска (дюрация): Низкий

Анализ успешно завершен!


In [2]:
from datetime import datetime, timedelta
from pys.data_collection.private_info import BASE_PATH
from pys.data_collection.bonds_kbd_pipeline import run_bond_selection_with_kbd

# Задаем временной интервал для анализа
end_date = datetime(2025, 4, 22)
start_date = datetime(2024, 1, 1)

# Запускаем пайплайн с использованием всех параметров
results = run_bond_selection_with_kbd(
    base_path=BASE_PATH,
    start_date=start_date,
    end_date=end_date,
    n_bonds=5,                     # Целевое число облигаций
    min_bonds=3,                   # Минимальное число облигаций
    weighting_strategy=None,       # Автоматический выбор на основе КБД
    strategy_profile='aggressive',
    kbd_yield_adjustment=-3.0,     # Снижаем порог доходности
    update_kbd_data=True,
    excluded_issuers=['ВТБ', 'Мечел'],  # Исключаемые эмитенты
    # included_issuers=['Газпром', 'РЖД'], # Включаемые эмитенты с приоритетом
    # required_rating='BB',          # Минимальный рейтинг
    # max_single_issuer_weight=0.25, # Ограничение веса одного эмитента (25%)
    output_format='all'            # Полная информация в результатах
)

print(f"\nУспешно создан портфель облигаций: {results['success']}")
if results['success']:
    print(f"Количество облигаций в портфеле: {len(results['portfolio'])}")
    print(f"Использованная стратегия взвешивания: {results['weighting_strategy']}")
    
    print("\nПути к сохраненным файлам:")
    for file_type, path in results['paths'].items():
        print(f"  {file_type}: {path}")


2025-04-26 01:25:25 - BondsKBDPipeline - INFO - Запуск пайплайна выбора облигаций с учетом КБД
2025-04-26 01:25:25 - KBDDownloader - INFO - KBDDownloader initialized
2025-04-26 01:25:25 - BondsKBDPipeline - INFO - Загружаем данные КБД с 2024-01-01 по 2025-04-22
2025-04-26 01:25:25 - KBDDownloader - INFO - Fetching KBD data from 2024-01-01 00:00:00 to 2025-04-22 00:00:00
2025-04-26 01:25:25 - KBDDownloader - INFO - Raw KBD data saved to /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/raw/raw_kbd_data_2024-01-01_to_2025-04-22.csv
2025-04-26 01:25:25 - KBDDownloader - INFO - Raw file contains 261 lines
2025-04-26 01:25:25 - KBDDownloader - INFO - First line contains 'zcyc', skipping it
2025-04-26 01:25:25 - KBDDownloader - INFO - Parsed CSV with columns: ['tradedate', 'tradetime', 'period_0.25', 'period_0.5', 'period_0.75', 'period_1.0', 'period_2.0', 'period_3.0', 'period_5.0', 'period_7.0', 'period_10.0', 'period_15.0', 'period_20.0', 'period_30.0']
2025-04-26 01:25

Успешно загружены актуальные данные КБД: 256 записей
КБД визуализация сохранена в: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/viz/kbd_curve_20250426_012525.png
Рекомендуемая стратегия взвешивания: equal - Равное взвешивание для защиты от инверсии кривой доходности

Параметры фильтрации облигаций:
  min_yield: 18.58
  max_yield: 56.58
  min_duration: 0.08000000000000002
  max_duration: 2.9699999999999993
  market_condition: inverted
  excluded_issuers: ['ВТБ', 'Мечел']
Обработано 6 облигаций

=== Рекомендованный облигационный портфель ===
      security_code   full_name  yield  duration_years  weight
16701  RU000A0JRU20     СЗКК 03  34.17        0.452500     0.2
16726  RU000A0JUAH8  РЖД-30 обл  20.63        0.510833     0.2
16696  KG000A3LSJ06  Кыргыз. 02  20.77        0.658333     0.2
16709  RU000A0JSGV0  РЖД-32 обл  20.96        1.135833     0.2
16712  RU000A0JT6B2   ВЭБ.РФ 19  19.91        0.922500     0.2

Портфель сохранен в /Users/aeshef/Documents/GitHub/

2025-04-26 01:25:26 - BondsProcessor - INFO - Visualization saved to /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/portfolios/bond_universe_20250426.png


Визуализация сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/portfolios/bond_universe_20250426.png


2025-04-26 01:25:26 - BondsProcessor - INFO - Portfolio analysis generated successfully
2025-04-26 01:25:26 - BondsKBDPipeline - INFO - Сформирован портфель из 5 облигаций (достаточно)
2025-04-26 01:25:26 - BondsKBDPipeline - INFO - Сводка результатов сохранена в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/results/bonds_results_20250426_012525.json
2025-04-26 01:25:26 - BondsKBDReport - INFO - Комбинированный отчет сохранен в /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/reports/bonds_kbd_report_20250426_012525.html
2025-04-26 01:25:26 - BondsKBDPipeline - INFO - Пайплайн выбора облигаций с учетом КБД завершен



=== Характеристики портфеля ===
Средневзвешенная доходность: 23.29%
Средневзвешенная дюрация: 0.74 лет
Уровень риска (дюрация): Низкий

Анализ успешно завершен!

Итоговые параметры фильтрации:
  min_yield: 18.58
  max_yield: 56.58
  min_duration: 0.08000000000000002
  max_duration: 2.9699999999999993
  market_condition: inverted
  excluded_issuers: ['ВТБ', 'Мечел']

Успешно создан портфель облигаций: True
Количество облигаций в портфеле: 5
Использованная стратегия взвешивания: equal

Пути к сохраненным файлам:
  report: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/reports/bonds_kbd_report_20250426_012525.html
  kbd_chart: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/viz/kbd_curve_20250426_012525.png
  portfolio_viz: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/portfolios/bond_universe_20250426.png
  results_summary: /Users/aeshef/Documents/GitHub/kursach/data/processed_data/BONDS/kbd/results/bonds_results_20250426